In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import scipy.io
import torch 
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import random
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, Normalize
from scipy.ndimage.interpolation import shift

In [12]:
device = torch.device('cuda:0')

def mat_reader(path_file, square):
    ecg = scipy.io.loadmat(path_file)
    ecg = {k: v for k, v in ecg.items() if k[0] != '_'}
    if square:
        return (ecg['val']).reshape(1, 60, 60)
    else:
        return (ecg['val']).reshape(1, 3600)


def x_y_data(data):
    np.random.shuffle(data)
    np.random.shuffle(data)

    y = []
    x = []
    for i in range(len(data)):
        k = data[i][0]
        y.append(k)
        m = data[i][1]
        x.append(torch.tensor(m).to(device))
    # print(y)
    y = torch.tensor(y).type(torch.cuda.LongTensor)
    x = torch.stack(x)
    
    return x, y


def data_loader(path_dir):
    data = []

    for _class, _cname in enumerate(os.listdir(path_dir)):
        frags_dir = os.path.join(path_dir, _cname)
        count = 0
        for _frags in os.listdir(frags_dir):
            frags_loc = os.path.join(frags_dir, _frags)
            x = mat_reader(frags_loc, False)
            y = _class
            count += 1
            data.append([x,y])

        if len(os.listdir(frags_dir)) < 284:
            inc_count = 284 - len(os.listdir(frags_dir))
            
            each_file = int(inc_count / len(os.listdir(frags_dir)))
            for _frags in os.listdir(frags_dir):
                for i in range(each_file):
                    count+=1
                    if i % 2 == 0:
                        alpha = np.copy(x)
                        alpha = np.roll(alpha, 2*i)
                        y = _class
                        data.append([alpha,y])
                    else:
                        alpha = np.copy(x)
                        alpha = np.roll(alpha, -2*i)
                        y = _class
                        data.append([alpha,y])
            print(_class, _cname, count)

    x, y = x_y_data(data)
    print(len(data))
    del data
    
    num_train = len(x)
    indices = list(range(num_train))
    split = int(np.floor(0.2 * num_train))

    np.random.shuffle(indices)

    valid_idx, test_idx, train_idx = indices[:split], indices[split:2 * split], indices[split:]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    train_loader = DataLoader(TensorDataset(x, y), batch_size=64, sampler=train_sampler)
    valid_loader = DataLoader(TensorDataset(x, y), batch_size=64, sampler=valid_sampler)
    test_loader = DataLoader(TensorDataset(x, y), batch_size=64, sampler=test_sampler)

    return train_loader, test_loader, valid_loader


class InitConv(nn.Module):
    def __init__(self, input_size, dropout, shape):  # 0  for linear, 1 for square
        super(InitConv, self).__init__()
        self.shape = shape
        self.relu = nn.ReLU()
        self.dropout_ = nn.Dropout(p=dropout)
        self.flatten = nn.Flatten()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

        if self.shape == 0:

            # Initial Convolution Model
            self.conv1d_1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=5, stride=2, bias=True)
            self.batch_norm_1d_1 = nn.BatchNorm1d(num_features=4, eps=0.00001, momentum=0.1)

            self.conv1d_2 = nn.Conv1d(in_channels=4, out_channels=16, kernel_size=4, stride=2, bias=True)
            self.batch_norm_1d_2 = nn.BatchNorm1d(num_features=16, eps=0.00001, momentum=0.1)

            self.conv1d_3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=True)
            self.batch_norm_1d_3 = nn.BatchNorm1d(num_features=32, eps=0.00001, momentum=0.1)

            self.conv1d_4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=4, stride=2, bias=True)
            self.batch_norm_1d_4 = nn.BatchNorm1d(num_features=32, eps=0.00001, momentum=0.1)

            self.linear_1d_1 = nn.Linear(in_features=7136, out_features=2048)
            self.linear_1d_2 = nn.Linear(in_features=2048, out_features=1024)
            self.linear_1d_3 = nn.Linear(in_features=1024, out_features=512)
            self.linear_1d_4 = nn.Linear(in_features=512, out_features=256)

            # Attention Model
            self.attention_linear_1d_1 = nn.Linear(in_features=256, out_features=256, bias=True)

        elif self.shape == 1:
            self.conv2d_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(2, 2), stride=(2, 2), bias=True)
            self.batch_norm_2d_1 = nn.BatchNorm2d(num_features=4, eps=0.00001, momentum=0.1)

            self.conv2d_2 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(2, 2), stride=(1, 2), padding=(2, 2),
                                      padding_mode="reflect")
            self.batch_norm_2d_2 = nn.BatchNorm2d(num_features=16, eps=0.00001, momentum=0.1)

            self.conv2d_3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2, 2), stride=(2, 1),
                                      padding=(2, 2), padding_mode="reflect")
            self.batch_norm_2d_3 = nn.BatchNorm2d(num_features=32, eps=0.00001, momentum=0.1)

    def forward(self, inputs):
        if self.shape == 0:
            t = inputs
            t = self.conv1d_1(t)
            t = self.batch_norm_1d_1(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.conv1d_2(t)
            t = self.batch_norm_1d_2(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.conv1d_3(t)
            t = self.batch_norm_1d_3(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.conv1d_4(t)
            t = self.batch_norm_1d_4(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.flatten(t)

            t = self.linear_1d_1(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.linear_1d_2(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.linear_1d_3(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.linear_1d_4(t)

            t = self.dropout_(t)

            alpha = t.detach().clone()

            alpha = self.tanh(alpha)
            alpha = alpha.view(alpha.shape[0], 1, alpha.shape[1])

            beta = self.attention_linear_1d_1(alpha)
            beta = self.softmax(beta)
            gamma = beta.detach().clone()

            for _ in range(16):
                beta = self.attention_linear_1d_1(alpha)
                gamma = torch.cat((gamma, beta), dim=1)
            gamma = self.softmax(gamma)

            for _ in range(17):
                t = t + gamma[:, _, :]

            t = self.relu(t)
            return t

        elif self.shape == 1:
            t = inputs
            t = self.conv2d_1(t)
            t = self.batch_norm_2d_1(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.conv2d_2(t)
            t = self.batch_norm_2d_2(t)
            t = self.relu(t)
            t = self.dropout_(t)

            t = self.conv2d_3(t)
            t = self.batch_norm_2d_3(t)
            t = self.relu(t)

            t = self.flatten(t)

            return t


class CnnBiLSTM1D(nn.Module):
    def __init__(self, dropout):
        super(CnnBiLSTM1D, self).__init__()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.dropout_ = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=1)

        # InitConv BLocks
        self.block_1 = InitConv(3600, 0.005, 0)
        self.block_2 = InitConv(3600, 0.005, 0)

        # Bi-Linear
        self.bilinear = nn.Bilinear(256, 256, 256)

        # Linear Layer
        self.linear_1 = nn.Linear(1024, 512, bias=True)
        self.linear_2 = nn.Linear(512, 256, bias=True)
        self.linear_3 = nn.Linear(256, 128, bias=True)
        self.linear_4 = nn.Linear(128, 64, bias=True)
        self.out = nn.Linear(64, 17, bias=True)

        # Conv Layer
        self.conv_1d_1 = nn.Conv1d(1, 8, kernel_size=4, stride=2, bias=True)

        # Batch Norm 1D
        self.batch_norm_1d_1 = nn.BatchNorm1d(8, eps=0.00001, momentum=0.1)

        # Bi-LSTM
        self.bi_lstm_1 = nn.LSTM(256, 512, num_layers=2, bias=True, dropout=0.005, bidirectional=True)
        self.bi_lstm_2 = nn.LSTM(1024, 1024, num_layers=2, bias=True, dropout=0.002, bidirectional=True)

    def forward(self, inputs):
        a = inputs.detach().clone()
        b = inputs.detach().clone()
        b = torch.flip(b, dims=[2])
        a = self.block_1(a)
        b = self.block_2(b)

        t = self.bilinear(a, b)

        t = t.view(t.shape[0], 1, t.shape[1])

        t, _ = self.bi_lstm_1(t)
        t = self.tanh(t)

#         t, _ = self.bi_lstm_2(t)
#         t = self.tanh(t)

        t = t.view(-1, t.shape[2])

        t = self.linear_1(t)
        t = self.relu(t)

        t = self.linear_2(t)
        t = self.relu(t)

        t = self.linear_3(t)
        t = self.relu(t)

        t = self.linear_4(t)
        t = self.relu(t)

        t = self.out(t)

        return t



network = CnnBiLSTM1D(0.005).to(device)

train_loader,test_loader, valid_loader= data_loader(r"../input/siemenshealthineersheatbeat-classfication/MLII")

print(len(train_loader))
print(len(test_loader))
print(len(valid_loader))



0 6 WPW 273
1 5 SVTA 273
2 2 APB 264
3 15 RBBBB 248
4 11 IVR 280
5 4 AFIB 270
6 7 PVC 266
7 1 NSR 283
8 13 Fusion 275
9 9 Trigeminy 273
10 3 AFL 280
11 12 VFL 280
12 14 LBBBB 206
13 16 SDHB 280
14 8 Bigeminy 275
15 17 PR 270
16 10 VT 280
4576
58
15
15


In [13]:
def get_num_correct(preds, labels):
    return preds.argmax(dim= 1).eq(labels).sum().item()

optimizer = optim.Adam(network.parameters(), lr=1e-4, weight_decay=1e-5)

valid_loss_min = np.Inf

for epoch in range(200):
    
    train_loss = 0
    train_correct = 0
    
    val_loss = 0
    val_correct = 0

    #####TRAIN DATA SET#####
    network.train()
    for batch in train_loader: # Get Batch
  
        labels, images = batch

        images = images.view(-1, 1, 3600).type(torch.cuda.FloatTensor)
        network.zero_grad()

        preds = network(images) # Pass Batch
        labels = labels.type(torch.cuda.LongTensor)
#         print(labels)
#         print(preds)
        loss = nn.CrossEntropyLoss()(preds, labels) # Calculate Loss

#         optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        train_loss += loss.item()
        train_correct += get_num_correct(preds, labels)
        

        
    #####Validation DATA SET#####
    
    for batch in valid_loader: # Get Batch

        labels, images = batch
        images = images.view(-1, 1, 3600).type(torch.cuda.FloatTensor)
        network.zero_grad()
        preds = network(images) # Pass Batch
        labels = labels.type(torch.cuda.LongTensor)
        loss = nn.CrossEntropyLoss()(preds, labels) # Calculate Loss
    
#         optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        val_loss += loss.item()
        val_correct += get_num_correct(preds, labels)
        
    print("Epoch", epoch, " ;Total Train correct:", train_correct, " ;Train loss:", train_loss)
    print("Total Validation correct:", val_correct ," ;Validation Loss:", val_loss)
    
    
    # save model if validation loss has decreased
    if val_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        val_loss))
        checkpoint = {'model': network,
              'state_dict': network.state_dict(),
              'optimizer' : optimizer.state_dict()}

        torch.save(checkpoint, 'NewDeep_Attentive_BiModel_99.89.pth')
        valid_loss_min = val_loss
    
    
    network.eval()
    with torch.no_grad():
        correct_all = 0
        total_all = 0
        for labels, images in test_loader:
            #             print(images.shape)
#             print(labels.shape)
            images = images.view(-1,1,3600).type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.FloatTensor)
#             print(labels.size(0))
            preds = network(images)
            correct = 0
            total = 0
            
            
            for i in range(labels.size(0)):
                act_label = labels[i] # act_label = 1 (index)
                pred_label = torch.argmax(preds[i]) # pred_label = 1 (index)
                
#                 print(act_label)
#                 print(pred_label)
                if(act_label == pred_label):
                    correct += 1
                total += 1
            total_all += total
            correct_all += correct
#         print(correct_all)
#         print(total_all)
        print('Test Accuracy of the model : {} %'.format(100*(correct_all/total_all))) 
    print('\n')

Epoch 0  ;Total Train correct: 241  ;Train loss: 164.04254484176636
Total Validation correct: 106  ;Validation Loss: 41.789501905441284
Validation loss decreased (inf --> 41.789502).  Saving model ...
Test Accuracy of the model : 13.77049180327869 %


Epoch 1  ;Total Train correct: 585  ;Train loss: 140.6969349384308
Total Validation correct: 157  ;Validation Loss: 32.25938153266907
Validation loss decreased (41.789502 --> 32.259382).  Saving model ...
Test Accuracy of the model : 20.655737704918035 %


Epoch 2  ;Total Train correct: 930  ;Train loss: 110.72570645809174
Total Validation correct: 356  ;Validation Loss: 25.167691588401794
Validation loss decreased (32.259382 --> 25.167692).  Saving model ...
Test Accuracy of the model : 38.5792349726776 %


Epoch 3  ;Total Train correct: 1428  ;Train loss: 90.81335020065308
Total Validation correct: 391  ;Validation Loss: 22.073504209518433
Validation loss decreased (25.167692 --> 22.073504).  Saving model ...
Test Accuracy of the model 

In [14]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

network_best = load_checkpoint('./NewDeep_Attentive_BiModel_99.89')
network_best= network_best.cuda()
print(network_best)


# accuracy_train= (train_correct/len(train))*100
# print('Train Accuracy=' + str(accuracy_train) + '%')


test_loss = 0.0
class_correct = list(0. for i in range(20))
class_total = list(0. for i in range(20))

network_best.eval() # prep model for evaluation

for target, data in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    data = data.view(-1, 1, 3600).type(torch.cuda.FloatTensor)
    output = network_best(data)
    # calculate the loss
    loss = F.cross_entropy(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class

    _, pred = torch.max(output, 1)
    # compare predictions to true label

    correct = np.squeeze(pred.eq(target.data.view_as(pred)))

    # calculate test accuracy for each object class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(20):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %.6f%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        continue

print('\nTest Accuracy (Overall): %.6f%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

CnnBiLSTM1D(
  (tanh): Tanh()
  (relu): ReLU()
  (dropout_): Dropout(p=0.005, inplace=False)
  (softmax): Softmax(dim=1)
  (block_1): InitConv(
    (relu): ReLU()
    (dropout_): Dropout(p=0.005, inplace=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (tanh): Tanh()
    (softmax): Softmax(dim=1)
    (conv1d_1): Conv1d(1, 4, kernel_size=(5,), stride=(2,))
    (batch_norm_1d_1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1d_2): Conv1d(4, 16, kernel_size=(4,), stride=(2,))
    (batch_norm_1d_2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1d_3): Conv1d(16, 32, kernel_size=(4,), stride=(2,))
    (batch_norm_1d_3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1d_4): Conv1d(32, 32, kernel_size=(4,), stride=(2,))
    (batch_norm_1d_4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear_1d_1): Linear(in_feature